<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0922/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

In [2]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [15]:
import numpy as np
# y: 0, 1, 2...
# Series.replace({"李白":0, ...})
# Series.unique()/Series.value_counts()
artists = train_df["作者"].unique()
# artist2idx = {}
# for i, a in enumerate(artists):
#    artist2idx[a] = i
artist2idx = {a:i for i, a in enumerate(artists)}
idx2artist = {i:a for i, a in enumerate(artists)}
y_train = np.array(train_df["作者"].replace(artist2idx))
y_test = np.array(test_df["作者"].replace(artist2idx))
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [32]:
# 物件導向: 自創一個你需要的型態
# 型態: 1. attribute(屬性) p1.height
#      2. method(專屬函式) "abc".replace("a", "b")
# __init__
# https://docs.python.org/zh-tw/3/reference/datamodel.html#basic-customization
class Person:
    def __init__(self, h, w):
        self.height = h
        self.weight = w

    def getbmi(self):
        return self.weight / (self.height / 100) ** 2

    def __str__(self):
        return "{}/{}".format(self.height, self.weight)

    def __eq__(self, other):
        return self.height == other.height and self.weight == other.weight

p1 = Person(175, 80)
bmi = p1.getbmi()
p2 = Person(175, 80)
# print -> str(p2) -> p2.__str__()
print(p2)
# p1 == p2 -> p1.__eq__(p2)
p1 == p2

175/80


True